# Notebook to VI rotation curve galaxies

In [2]:
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle

In [3]:
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/' # need to make a cache in pscratch? vi_rotation_curves?
vi_dir = '/pscratch/sd/j/jjpim/visual_inspection/' # need to make in pscratch?

In [4]:
def get_cutout(SGA_ID, ra, dec, size, cache_dir, verbose=False):
    '''Grab and cache legacy survey cutouts.
    
    Parameters
    ----------
    SGA_ID : int
        SGA ID for galaxy.
    ra : float
        Right ascension (degrees).
    dec : float
        Declination (degrees).
    size : float
        size of cutout.
    cache_dir : string
        cache location
    verbose : bool
        Add some status messages if true.
        
    Returns
    -------
    img_name : str
        Name of JPG cutout file written after query.
    w : astropy.wcs.WCS
        World coordinate system for the image.
    '''
    
    # Either load an existing image or download a cutout.
    img_name = cache_dir + '{}.jpg'.format(SGA_ID)
    
    
    if os.path.exists(img_name):
        if verbose:
            print('{} exists.'.format(img_name))

    else:
        img_url = 'https://www.legacysurvey.org/viewer/cutout.jpg?ra={}&dec={}&zoom=14&layer=ls-dr9&size={}&sga'.format(ra, dec, size)
        if verbose:
            print('Get {}'.format(img_url))
            
        with open(img_name, 'wb') as handle: 
            response = requests.get(img_url, stream=True) 
            if not response.ok: 
                print(response) 
            for block in response.iter_content(1024): 
                if not block: 
                    break 
                handle.write(block)
                
    # Set up the WCS.
    wcs_input_dict = {
        'CTYPE1': 'RA---TAN',
        'CUNIT1': 'deg',
        'CDELT1': -0.262/3600,
        'CRPIX1': size/2 + 0.5,
        'CRVAL1': ra,
        'NAXIS1': size,
        'CTYPE2': 'DEC--TAN',
        'CUNIT2': 'deg',
        'CDELT2': 0.262/3600,
        'CRPIX2': size/2 + 0.5,
        'CRVAL2': dec,
        'NAXIS2': size
    }
    w = WCS(wcs_input_dict)
    
    return img_name, w

In [7]:
# load fuji table
tf_fuji = Table.read(data_dir + 'SV/desi_pv_tf_fuji_healpix_rotcurve_EOA.fits')
tf_fuji[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136,1.0,0.012960487467698754,0.3300000056318236,0.0,0.0
1092744369930240,194.338458724402,27.4918902690326,10378,sv3,0.0264170223697961,0.00010139452689994,0,9.53278421035066,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.33845872440244,27.491890269032595,1.0,0.012960487467685656,0.33000000563149007,0.0,0.0
1092744374124545,194.377858465028,27.5098100780282,10378,sv3,0.211332646769145,6.68535116703737e-05,4,3.73989077657461,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3778584650283,27.509810078028195,1.0,0.026313716979861984,0.6700000114340222,0.0,0.0


In [8]:
# load iron table
tf_iron = Table.read(data_dir + 'Y1/desi_pv_tf_iron_healpix_rotcurve_EOA.fits')
tf_iron[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
2852147603439621,198.369130660983,36.5372037049171,10475,main,0.815976335547845,7.38513168100107e-05,4,0.128754377365112,iron/healpix/main/dark/104/10475/redrock-main-dark-10475.fits,EXT,649377,198.36913066098333,36.537203704917076,0.0,0.0567420462925582,0.599999979158402,0.0,1.0
2399148812795907,198.371733180003,36.4994335406917,10475,main,1.11088784970434,7.48767797671894e-05,4,7.9473560154438,iron/healpix/main/bright/104/10475/redrock-main-bright-10475.fits,EXT,649377,198.37173318000336,36.499433540691676,0.0,0.09457007715465947,0.9999999652681886,0.0,1.0
2399382443917318,184.845242475328,49.8157304793777,10995,main,1.14739342108157,0.000146302276719084,4,2.56771463155746,iron/healpix/main/bright/109/10995/redrock-main-bright-10995.fits,EXT,1008911,184.84524247532795,49.81573047937771,0.0,0.024841917763806447,0.9999999684347345,0.0,0.0
2399634072797192,184.341289722203,70.8283725474297,11965,main,1.51703376230705,6.28979649962091e-05,4,4.76254060305655,iron/healpix/main/bright/119/11965/redrock-main-bright-11965.fits,EXT,241234,184.34128972220284,70.82837254742968,0.0,0.026555350833845817,0.8000000542794348,0.0,0.0
2852141710442505,123.256011148025,36.2652948002806,6448,main,0.00787379494184006,3.4714052819995e-05,0,22.1719104201402,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.25601114802525,36.26529480028061,0.0,0.015878717350385107,0.40000000619171416,0.0,0.0


In [9]:
# load SGA table
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')
SGA_dict = {}
for i in range(len(SGA)):
    SGA_dict[SGA['SGA_ID'][i]] = i
SGA[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.54443750000002,6.9345944,0.4017908,

In [10]:
# list of rotation curve galaxies
tf_mastertable = vstack([tf_fuji,tf_iron])
rot_curve_gals = np.unique(tf_mastertable['SGA_ID'][tf_mastertable['ROT_CURVE']==1])

In [ ]:
# for each rotation curve galaxy, grab cut out, draw fibers on image, and save image
for sga_id in rot_curve_gals:
    
    targ_list = tf_mastertable[tf_mastertable['SGA_ID'] == sga_id]
    if np.min(targ_list['SKY_FIBER_DIST_R26']) < 0.001:
        center_target = targ_list[np.argmin(targ_list['SKY_FIBER_DIST_R26'])]['TARGETID']
        z = targ_list[np.argmin(targ_list['SKY_FIBER_DIST_R26'])]['Z']
        
    else:
        continue
    
    ra, dec = float(SGA['RA'][SGA_dict[sga_id]]), float(SGA['DEC'][SGA_dict[sga_id]])
    
    # D26 in arcmin
    d26 = SGA['D26'][SGA_dict[sga_id]]
    

    npix = np.minimum(int(2 * d26*60/0.262), 512)
    

    img_file, wcs = get_cutout(sga_id, ra, dec, size=npix, cache_dir=cache_dir,verbose=True)
    img = mpl.image.imread(img_file)

    fig1 = plt.figure(figsize=(7,5))

    ax = fig1.add_subplot(111, projection=wcs)
    ax.imshow(np.flip(img, axis=0))
    ax.set(xlabel='ra', ylabel='dec')
    ax.text(int(0.02*npix), int(0.85*npix), 'SGA_ID: {}\n$z={{{:.4f}}}$'.format(sga_id, z), fontsize=9, color='yellow')
    overlay = ax.get_coords_overlay('icrs')
    overlay.grid(color='white', ls='dotted');

    # Add the location of the DESI fibers.
    # SDSS fibers are 2" diameter, DESI is 107 um with 70 um/" plate scale.
    r1 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                         edgecolor='black', facecolor='none', alpha=0.8, lw=3,
                         transform=ax.get_transform('icrs'))
    r2 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                         edgecolor='red', facecolor='none', alpha=0.8, lw=2,
                         transform=ax.get_transform('icrs'))
    ax.add_patch(r1)
    ax.add_patch(r2)

    for targ in targ_list:
        ra, dec = targ['RA'], targ['DEC']
        
        edgecolor2 = 'orange'

        # Add the location of the DESI fibers.
        # SDSS fibers are 2" diameter, DESI is 107 um with 70 um/" plate scale.
        r1 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                             edgecolor='lightcoral', facecolor='none', alpha=1, lw=3,
                             transform=ax.get_transform('icrs'))
        r2 = SphericalCircle((ra * u.deg, dec * u.deg), (107./70) * u.arcsec,
                             edgecolor=edgecolor2, facecolor='none', alpha=0.8, lw=2,
                             transform=ax.get_transform('icrs'))
        ax.add_patch(r1)
        ax.add_patch(r2)
        
        ax.text(ra, dec, str(targ['TARGETID']), transform=ax.get_transform('icrs'), color='white', fontsize=6)
    
    fig1.subplots_adjust(top=0.85, right=0.85, bottom=0.15, left=0.15)
    
    fig1.savefig(vi_dir + '{}.png'.format(sga_id), dpi=120)
    
    fig1.clear()
    plt.close(fig1)


In [13]:
# we want to visually inspect rotation curve galaxies that have EOA targets
EOA_gals = np.unique(tf_mastertable[np.logical_and(tf_mastertable['EOA'] == 1, tf_mastertable['ROT_CURVE']==1)]['SGA_ID'])
print(EOA_gals)

 SGA_ID
-------
  81587
 526710
 608015
 696291
 766672
 882260
 977015
1065602
1304908
1361363
2000055


In [14]:
np.where(SGA['SGA_ID']==81587)

(array([21528]),)

In [15]:
SGA[21528]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
81587,SGA-2020 81587,NGC5964,55637,234.40090950000004,5.9740148,SBcd,147.0,3.4197948,0.7498942,0.004826339,24.911482,13.613,False,LEDA-20181114,29237,NGC5964,1,True,234.40090950000004,5.9740148,3.4197948,2343p060,234.40098578623957,5.973890372677745,5.0586667,SB26,146.60901,0.76359373,234.40091051554066,5.9739991397431425,139.96718,61.725025,60.566208,57.804718,23.219322,36.98816,54.375183,73.22268,89.9901,106.54655,121.93921,136.91577,151.76,14.373638,13.728431,13.220966,13.750148,13.111622,12.617482,13.230628,12.6180105,12.134496,12.901384,12.298157,11.821981,12.705964,12.109203,11.640028,12.58583,11.990436,11.527844,12.513165,11.920093,11.463872,12.462504,11.871767,11.419979,12.430036,11.840087,11.392938,0.10281313,0.098655805,0.10543129,0.10789143,0.117761284,0.06803115,0.1775874,0.20067601,0.29327112,0.0041126143,0.0040354356,0.0041690753,0.0025483703,0.0025393318,0.002642904,0.0016863007,0.0017121384,0.0018007569,0.0013016955,0.0013189712,0.0013993608,0.0011104371,0.0011251417,0.0012071129,0.0010096927,0.0010217316,0.0011081086,0.00095429376,0.0009658115,0.0010585015,0.0009171809,0.00092964846,0.0010282589,0.0008961264,0.0009082605,0.0010137698,12.33269,0.4334786,1727.5459,3.2482514,47.456753,11.744617,0.4189626,1741.7992,3.2467062,19.600035,11.315926,0.38606414,2492.3994,3.434185,17.1178,0


In [26]:
SGA['SGA_ID']==608015

array([False, False, False, ..., False, False, False])

In [78]:
SGA[SGA['SGA_ID']== 1065602]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
1065602,SGA-2020 1065602,NGC4030,37845,180.0983565,-1.100072,Sbc,21.4,3.810658,0.7030723,0.00487604,22.68248,11.149,False,LEDA-20181114,391650,NGC4030,1,True,180.0983565,-1.100072,3.810658,1801m010,180.0984725183874,-1.1002054875552552,5.0754147,SB26,43.987,0.7817526,180.09867775445406,-1.1004246487750595,125.81903,31.709398,28.159693,24.553022,65.23249,76.8927,88.43179,100.083176,111.4049,122.097275,132.54459,142.2664,152.26244,11.1092205,10.404146,9.806159,11.021408,10.324484,9.733617,10.959452,10.268005,9.68207,10.9179535,10.229303,9.646551,10.890883,10.204431,9.623922,10.872638,10.187836,9.60895,10.860583,10.176522,9.599226,10.853263,10.16968,9.593003,10.848434,10.165096,9.588847,0.06336082,0.10379152,0.096198544,0.12710512,0.16125621,0.11254954,0.21445172,0.18153988,0.26804462,0.0016041694,0.0016298264,0.0020912334,0.0014859528,0.0015198538,0.0019616652,0.0014078751,0.0014458095,0.0018726526,0.0013578547,0.0013971945,0.0018136896,0.0013257687,0.0013665466,0.0017769423,0.0013027375,0.0013442624,0.001749664,0.0012905708,0.0013328118,0.0017376026,0.0012822334,0.0013246832,0.0017278544,0.0012767481,0.001319236,0.0017213816,10.830426,0.15890045,17807.633,4.384688,0.825696,10.14877,0.13953987,21801.98,4.4441447,0.38816977,9.575387,0.11971263,34007.87,4.6197586,0.081386246,0


In [79]:
# inspect all the necessary galaxies, 
# if you find a galaxy with bad target for rotation curve analysis, grab all observations for that galaxy
tf_mastertable[tf_mastertable['SGA_ID'] == 1065602]
# identify which target is bad by target id and add to the list in the cell below

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
1083283966263312,180.088777358876,-1.11024777720183,25599,sv3,0.00436762087876588,1.42837672240012e-06,0,15897.7449971626,fuji/healpix/sv3/dark/255/25599/redrock-sv3-dark-25599.fits,EXT,1065602,180.08877735887648,-1.1102477772018335,1.0,0.013957389713373361,0.32999998593202395,1.0,1.0
1083283966263313,180.108167612631,-1.09016316641461,25599,sv3,0.0053534602140923,4.45334000484803e-06,0,4136.92135445221,fuji/healpix/sv3/dark/255/25599/redrock-sv3-dark-25599.fits,EXT,1065602,180.10816761263058,-1.090163166414607,1.0,0.013957389713391206,0.3299999859324459,1.0,1.0
1070089826729999,180.078788337759,-1.1205943454118,25599,sv3,0.746080375788651,7.05932471404864e-05,0,12.3854653686285,fuji/healpix/sv3/bright/255/25599/redrock-sv3-bright-25599.fits,EXT,1065602,180.07878833775885,-1.1205943454118001,1.0,0.02833773063017716,0.6699999714376246,1.0,1.0
1083283966263311,180.078788337759,-1.1205943454118,25599,sv3,0.00464639799063975,3.19435285822499e-05,0,31.9120003114913,fuji/healpix/sv3/dark/255/25599/redrock-sv3-dark-25599.fits,EXT,1065602,180.07878833775885,-1.1205943454118001,1.0,0.02833773063017716,0.6699999714376246,1.0,1.0
1083283966263314,180.118156429974,-1.07981649987604,25599,sv3,0.00535718577838891,1.59550379891168e-05,0,249.207345602888,fuji/healpix/sv3/dark/255/25599/redrock-sv3-dark-25599.fits,EXT,1065602,180.11815642997368,-1.0798164998760402,1.0,0.02833773063019543,0.6699999714380566,1.0,1.0
1070083786932227,180.069093043901,-1.13063657023067,25599,sv3,0.508852243075155,0.000176351857769613,4,0.803055022843182,fuji/healpix/sv3/bright/255/25599/redrock-sv3-bright-25599.fits,TFT,1065602,180.06909304390072,-1.1306365702306744,1.0,0.04229512034355193,0.9999999573696817,1.0,1.0
1083277926465539,180.069093043901,-1.13063657023067,25599,sv3,0.609555769981184,9.6756850868172e-05,4,4.17820824682713,fuji/healpix/sv3/dark/255/25599/redrock-sv3-dark-25599.fits,TFT,1065602,180.06909304390072,-1.1306365702306744,1.0,0.04229512034355193,0.9999999573696817,1.0,1.0
1070089826730003,180.127851393537,-1.06977411567805,25599,sv3,1.37914220539999,8.90974056076415e-05,4,3.03583170473576,fuji/healpix/sv3/bright/255/25599/redrock-sv3-bright-25599.fits,TFT,1065602,180.12785139353724,-1.0697741156780503,1.0,0.042295120343574265,0.9999999573702099,1.0,1.0
1083283966263315,180.127851393537,-1.06977411567805,25599,sv3,0.41132741165799,6.08823852491685e-05,4,3.33358387649059,fuji/healpix/sv3/dark/255/25599/redrock-sv3-dark-25599.fits,TFT,1065602,180.12785139353724,-1.0697741156780503,1.0,0.042295120343574265,0.9999999573702099,1.0,1.0


In [67]:
# use to determine if specific target is within desired set
tf_mastertable[tf_mastertable['TARGETID']==  2399058668814336]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64


In [ ]:
# if there is a target not suitable for rotation curve analysis,
#change 'ROT_CURVE' to 0 for that target in the tables 
# (targets can be in both fuji and iron)

targetids = [2403461840764930, 2407864991744014, 2847669642854414]
for targetid in targetids:
    try:
        tf_fuji[tf_fuji['TARGETID']['ROT_CURVE'] == targetid] = 0
    except:
        print(targetid, ' not in fuji')
    
    try:
        tf_iron[tf_iron['TARGETID']['ROT_CURVE'] == targetid] = 0
    except:
        print(targetid, ' not in iron')

In [42]:
# additional targets that need to inspected, some of these may be elliptical galaxies which we can't use
# for rotation curve analysis
EXT_centers = tf_mastertable[np.logical_and(tf_mastertable['PVTYPE'] == 'EXT', tf_mastertable['SKY_FIBER_DIST_R26'] < 0.001)]
EXT_centers[EXT_centers['ROT_CURVE'] == 1]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
39627865758633464,150.11281896177,3.37442274219938,27345,sv3,0.00687190356978907,1.5141461957797e-06,0,24405.1790848412,fuji/healpix/sv3/bright/273/27345/redrock-sv3-bright-27345.fits,EXT,1225398,150.1128189617695,3.3744227421993793,1.0,0.0,0.0,1.0,0.0
39627770296276005,219.956999318774,-0.718640392418429,25933,sv3,0.00585538786570846,4.19501768104605e-06,0,2858.02824933385,fuji/healpix/sv3/bright/259/25933/redrock-sv3-bright-25933.fits,EXT,166290,219.95699931877425,-0.7186403924184287,1.0,0.0,0.0,1.0,0.0
39632981425390961,222.111381963082,34.9980057574604,9187,main,0.0294233178741356,1.01512926413074e-05,0,21237.1530243214,iron/healpix/main/bright/91/9187/redrock-main-bright-9187.fits,EXT,492092,222.11138196308235,34.998005757460405,0.0,0.0,0.0,1.0,0.0
39632941206208838,216.019898540707,33.0474110189656,10501,main,0.00655488909124642,3.64638274065069e-06,0,133962.636167373,iron/healpix/main/bright/105/10501/redrock-main-bright-10501.fits,EXT,617559,216.0198985407066,33.04741101896559,0.0,0.0,0.0,1.0,0.0
39627726562266974,133.158364109104,-2.60312501271759,40954,main,0.00544621749690199,4.982629598787e-06,0,42836.996128482,iron/healpix/main/bright/409/40954/redrock-main-bright-40954.fits,EXT,1324787,133.15836410910384,-2.6031250127175887,0.0,0.0,0.0,1.0,0.0
39627770296276005,219.956999318774,-0.718640392418429,25933,main,0.00585642761793563,3.18737529132276e-06,0,4559.97703461154,iron/healpix/main/bright/259/25933/redrock-main-bright-25933.fits,EXT,166290,219.95699931877425,-0.7186403924184287,1.0,0.0,0.0,1.0,0.0
39627770296276005,219.956999318774,-0.718640392418429,25933,sv3,0.00585552930485212,4.18669924707662e-06,0,2885.67530876072,iron/healpix/sv3/bright/259/25933/redrock-sv3-bright-25933.fits,EXT,166290,219.95699931877425,-0.7186403924184287,1.0,0.0,0.0,1.0,0.0
39627865758633464,150.11281896177,3.37442274219938,27345,sv3,0.00687091117279877,1.49978640820426e-06,0,25621.2598910853,iron/healpix/sv3/bright/273/27345/redrock-sv3-bright-27345.fits,EXT,1225398,150.1128189617695,3.3744227421993793,1.0,0.0,0.0,1.0,0.0
39627683235107151,68.2748504845867,-4.29723311321309,22746,main,0.0144011180784538,6.45771577100659e-06,0,9619.54782551713,iron/healpix/main/bright/227/22746/redrock-main-bright-22746.fits,EXT,317222,68.27485048458668,-4.297233113213093,0.0,0.0,0.0,1.0,0.0
